In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
from datetime import datetime,date
DW_EFF_DT=date.today()

In [ ]:
data_url='https://www.mohfw.gov.in/'

In [ ]:
html_text=requests.get(data_url).text

In [ ]:
soup=BeautifulSoup(html_text,'html5lib')

In [ ]:
soup

In [ ]:
table_content=soup.find_all('table', class_='table table-striped')

In [ ]:
table_content[:]

In [4]:
import pandas as pd

In [ ]:
state_df=pd.read_html(str(table_content[:]))

In [ ]:
state_df[0]

In [ ]:
state_df[0]['DW_EFF_DT']=DW_EFF_DT

In [ ]:
state_dly_stats=state_df[0][['DW_EFF_DT','Name of State / UT','Total Confirmed cases (Including 76 foreign Nationals)','Cured/Discharged/Migrated']]

In [ ]:
state_dly_stats[:-2]

In [ ]:
state_wise_cnt=state_dly_stats[:-2]

In [ ]:
state_wise_cnt

In [5]:
import pyodbc #pyodbc is library used to connect to MSSQL


In [6]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=@@@;"
                      "Database=@@@;"
                      "Trusted_Connection=yes;")

In [ ]:
# code to insert data from dataframe to MSSQL.

for indx,row in state_wise_cnt.iterrows():
    with cnxn.cursor() as crsr:
        crsr.execute("INSERT INTO IDW.dbo.COVID_DLY_T(DW_EFF_DT,STATES,TOTL_CNFM_CS_IN,CURED_DISCHG) values(?,?,?,?)",row["DW_EFF_DT"],row["Name of State / UT"],row["Total Confirmed cases (Including 76 foreign Nationals)"],row["Cured/Discharged/Migrated"])

In [7]:
covid_dly_t=pd.read_sql_query("select * from IDW.dbo.COVID_DLY_T",cnxn)

In [ ]:
#this code is to generate final excel sheet which would be ingested to tableau
covid_dly_t.to_excel('COVID-Report.xlsx',index=False)

In [ ]:
crsr.commit()
crsr.close()
cnxn.close()

Chart plotting

In [9]:
covid_mi=covid_dly_t.set_index(['DW_EFF_DT','STATES'])

In [10]:
covid_mi_1=(covid_mi.unstack(level=1).fillna(0).astype(int))

In [12]:
from ipywidgets import interact

In [13]:
states_name=covid_dly_t['STATES'].drop_duplicates()

In [19]:
@interact(State=states_name)
def covid_plot(State='Delhi'):
    covid_mi_1.loc[:,'TOTL_CNFM_CS_IN'].loc[:,State].plot()
    covid_mi_1.loc[:,'CURED_DISCHG'].loc[:,State].plot()


interactive(children=(Dropdown(description='State', index=5, options=('Andaman and Nicobar Islands', 'Andhra P…